In this notebook I take the data that was produced in the descriptive analysis section and fit a generalized radiation model (GRM) to the data.

In [1]:
import pandas as pd
pre_kincaid = pd.read_csv('../data/clean/pre_kincaid.csv')
pre_kincaid = pre_kincaid[pre_kincaid['county_o'] != pre_kincaid.county_d]
pre_kincaid = pre_kincaid.merge(pre_kincaid.groupby('geoid_o')[['pop_flows']].sum().rename({'pop_flows':'outflow'}, axis=1), on = 'geoid_o', how='left')
pre_kincaid['coord_o'] = pd.Series(pre_kincaid[['lat_o','lng_o']].itertuples(index=False, name=None))
pre_kincaid['coord_d'] = pd.Series(pre_kincaid[['lat_d','lng_d']].itertuples(index=False, name=None))

This is a class to fit and predict with a generalized radiation model

In [2]:
from geopy.distance import geodesic
import numpy as np
# from scipy.optimize import curve_fit
from scipy.optimize import minimize
from sklearn.preprocessing import FunctionTransformer

class GeneralizedRadiationModel:
    def __init__(self):
        self.X_cols_i = None
        self.X_cols_j = None
        self.weights = None

    def point_in_circle(self, origin, destination, point):
        origin_destination_distance = geodesic(origin, destination).meters
        origin_point_distance = geodesic(origin, point).meters
        return ((origin_point_distance <= origin_destination_distance) and (point != origin) and (point != destination))

    def calculate_v_ij(self, df):
        v_ij_pre_weight = df.apply(lambda row: df.loc[df.coord_d.apply(lambda x: self.point_in_circle(row['coord_o'], row['coord_d'], x)), self.X_cols_j].sum(), axis=1)
        return v_ij_pre_weight

    def radiation_model(self, T_i, U_i, U_j, v_ij, weights):

        weighted_U_i = U_i @ weights
        weighted_U_j = U_j @ weights
        weighted_v_ij = v_ij @ weights

        return T_i * (weighted_U_i * weighted_U_j) / ((weighted_U_i + weighted_v_ij) * (weighted_U_i + weighted_U_j + weighted_v_ij))

    def fit(self, df, coord_cols, X_cols_i, X_cols_j, T_i_col, T_ij_col):
        self.X_cols_i = X_cols_i
        self.X_cols_j = X_cols_j
        self.outflow_col = T_i_col

        coord_df = df[coord_cols]
        X_i = df[X_cols_i].values
        X_j = df[X_cols_j].values
        T_i = df[T_i_col].values
        T_ij = df[T_ij_col].values

        v_ij_pre_weight = self.calculate_v_ij(df)
        self.v_ij = v_ij_pre_weight

        def error_function(weights, *args):
            fitted_T_ij = self.radiation_model(T_i, X_i, X_j, v_ij_pre_weight, weights)
            residuals = fitted_T_ij - T_ij
            return np.sum(residuals**2)

        # Initial weight values (can be adjusted based on your requirements)
        initial_weights = np.ones(len(X_cols_j))

        self.optimal_result = minimize(error_function, x0 =  initial_weights, method = 'Nelder-Mead')

        self.weights = self.optimal_result.x

    def predict(self, df):
        v_ij = self.calculate_v_ij(df)
        return self.radiation_model(df[self.outflow_col].values, df[self.X_cols_i].values, df[self.X_cols_j].values, v_ij.values, self.weights)

# Define the logistic z-score transformation function
def logistic_zscore(x):
    x_mean = np.mean(x)
    x_std = np.std(x)
    transformed_x = 1 / (1 + np.exp(-(x - x_mean) / x_std))
    return transformed_x

# Create a FunctionTransformer with the logistic z-score transformation function
scaler = FunctionTransformer(logistic_zscore)

Before fitting, let's scale the x variables.

In [3]:
pre_kincaid[['pop_o','pop_d']] = scaler.fit_transform(pre_kincaid[['pop_o','pop_d']])

In [ ]:
grm = GeneralizedRadiationModel()
grm.fit(pre_kincaid, ['coord_o','coord_d'], ['pop_o', 'eigen_centrality_o'], ['pop_d','eigen_centrality_d'], 'outflow','pop_flows')

predicted = grm.predict(pre_kincaid)
pre_kincaid['predicted_flow'] = predicted
pre_kincaid[['v_ij_pop', 'v_ij_centrality']] = grm.v_ij

In [ ]:
pre_kincaid.plot.scatter(x='predicted_flow', y='pop_flows', logx=True, logy=True)

In [ ]:
grm_2 = GeneralizedRadiationModel()
grm_2.fit(pre_kincaid, ['coord_o','coord_d'], ['pop_o'], ['pop_d'], 'outflow','pop_flows')

predicted = grm_2.predict(pre_kincaid)
pre_kincaid['predicted_flow'] = predicted
pre_kincaid[['v_ij_pop']] = grm_2.v_ij

In [ ]:
pre_kincaid.plot.scatter(x='predicted_flow', y='pop_flows', logx=True, logy=True)

In [ ]:
np.round(grm.optimal_result['fun']/grm_2.optimal_result['fun'],3)

The inclusion of eigenvector centrality improves the RMSE by 5% relative to excluding it.

- Model of traffic flows trained on the flow data from prior to the fire
- Model of traffic flows trained on the flow data during the fire

In [5]:
# long lats
kincaid_coords = (38.792458, -122.780053)
czu_coords = (37.17162, -122.22275)
august_coords = (39.776, -122.673)

In [6]:
path = '../data/clean/'
during_kincaid = pd.read_csv(path + 'during_kincaid_merged.csv')
pre_kincaid = pd.read_csv(path + 'pre_kincaid_merged.csv')
pre_czu = pd.read_csv(path + 'pre_czu_merged.csv')
during_czu = pd.read_csv(path + 'during_czu_merged.csv')
during_august = pd.read_csv(path + 'during_august_merged.csv')
pre_august = pd.read_csv(path + 'pre_august_merged.csv')

Convert to coordinates.

In [7]:
def create_coord(df):
    df = df.copy()
    df['coord_o'] = pd.Series(df[['lat_o','lng_o']].itertuples(index=False, name=None))
    df['coord_d'] = pd.Series(df[['lat_d','lng_d']].itertuples(index=False, name=None))
    return df

dataframes = {
        'during_august': during_august,
    'pre_august': pre_august,
    'during_kincaid': during_kincaid,
    'pre_kincaid': pre_kincaid,
    'pre_czu': pre_czu,
    'during_czu': during_czu
}

for key in dataframes:
    dataframes[key] = create_coord(dataframes[key])


Include outflows:

In [8]:
def create_outflows(df):
    df = df.copy()
    df = df.merge(df.groupby('geoid_o')[['pop_flows']].sum().rename({'pop_flows':'outflow'}, axis=1), on = 'geoid_o', how='left')
    return df

for key in dataframes:
    dataframes[key] = create_outflows(dataframes[key])

Calculate distance to fire.

In [9]:
def create_distance_to_fire(df, fire_coordinates):
    df = df.copy()
    df['distance_to_fire_o'] = df.coord_o.apply(lambda x: geodesic(x, fire_coordinates).kilometers)
    df['distance_to_fire_d'] = df.coord_d.apply(lambda x: geodesic(x, fire_coordinates).kilometers)
    return df

for key in ['pre_kincaid','during_kincaid']:
    dataframes[key] = create_distance_to_fire(dataframes[key], kincaid_coords)

for key in ['pre_czu','during_czu']:
    dataframes[key] = create_distance_to_fire(dataframes[key], czu_coords)

for key in ['pre_august','during_august']:
    dataframes[key] = create_distance_to_fire(dataframes[key], august_coords)

Scale the data:

In [10]:
for key in dataframes:
    dataframes[key][['pop_o','pop_d','distance_to_fire_o','distance_to_fire_d']] = dataframes[key][['pop_o','pop_d','distance_to_fire_o','distance_to_fire_d']].apply(logistic_zscore)

Include whether or not a path is blocked.

Fit models on all the different datasets:

In [11]:
results = dict()

for key in dataframes:
    results.update({key: dict()})
    results[key].update({'model' : GeneralizedRadiationModel()})
    print(key)
    results[key]['model'].fit(dataframes[key], ['coord_o','coord_d'], ['pop_o', 'eigen_centrality_o', 'distance_to_fire_o'], ['pop_d', 'eigen_centrality_d', 'distance_to_fire_d'], 'outflow','pop_flows')

during_august
